# LDA classification of CyTOF

#### Verifying results of Predicting   cell   types   in   single   cell   mass   cytometry data

In [4]:
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
from sklearn.metrics import f1_score
import os
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


### Sample of data
28 features over a large number of cells per CSV

In [7]:
sample = pd.read_csv("../Data/PANORAMA/Samples/Samples01.csv", header=None)
markers = ['Ter119','CD45.2','Ly6G','IgD','CD11c','F480','CD3','NKp46','CD23',
            'CD34','CD115','CD19','120g8','CD8','Ly6C','CD4','CD11b','CD27','CD16_32',
            'SiglecF','Foxp3','B220','CD5','FceR1a','TCRgd','CCR7','Sca1','CD49b','cKit',
            'CD150','CD25','TCRb','CD43','CD64','CD138','CD103','IgM','CD44','MHCII']
sample.columns = markers
sample.head(10)

,Ter119,CD45.2,Ly6G,IgD,CD11c,F480,CD3,NKp46,CD23,CD34,...,CD150,CD25,TCRb,CD43,CD64,CD138,CD103,IgM,CD44,MHCII
0,-0.30416,1.31230,-0.286970,-0.20859,1.402700,1.20650,-0.213720,-0.054569,-0.239330,-0.282240,...,0.874740,-0.016584,-0.099003,5.720300,0.66856,-0.098138,-0.229340,-0.246720,3.34600,-0.050778
1,-0.33094,0.19375,-0.009644,-0.33023,-0.339830,1.32740,-0.321190,-0.294260,-0.215410,-0.292410,...,-0.304200,-0.318980,-0.239430,0.297110,0.43763,0.023776,-0.303940,0.202910,3.99780,0.344630
2,-0.30554,1.74500,1.051400,-0.27056,-0.143130,-0.30412,-0.199820,-0.261000,-0.253090,-0.341760,...,-0.228320,-0.315390,-0.238720,-0.250980,-0.33615,-0.189390,-0.137230,5.108300,0.39405,3.666300
3,-0.34303,2.01170,-0.172230,-0.33583,-0.342000,2.00310,-0.284280,-0.186040,0.300580,-0.290580,...,-0.263700,-0.222530,-0.266030,-0.155270,2.76790,-0.239380,-0.023883,-0.034218,4.48520,0.701710
4,-0.19893,1.81050,-0.300430,-0.32335,-0.342810,0.60875,-0.313480,0.570740,-0.244890,-0.096765,...,-0.296520,-0.136830,1.123000,3.555300,0.69628,0.102030,-0.016809,-0.264990,4.78600,0.606260
5,-0.21946,0.61753,-0.298550,-0.22917,-0.296820,1.09270,-0.257650,-0.335930,-0.295930,0.409980,...,-0.280840,0.278570,-0.332040,0.189010,2.29020,0.229340,-0.209560,0.376750,3.79260,0.669650
6,-0.28746,0.95229,0.018152,-0.31537,-0.139070,2.22740,-0.197400,-0.148140,-0.239470,0.717520,...,-0.302030,-0.242370,-0.294070,0.092166,0.21569,-0.320490,0.559120,-0.047981,3.18400,0.616270
7,-0.32242,0.90390,0.292450,0.67402,0.033029,1.97480,-0.331360,0.191630,-0.062204,1.518700,...,1.163300,-0.225190,-0.151240,4.113800,-0.12754,-0.099417,-0.270240,-0.315430,4.87170,0.935400
8,-0.26139,0.29810,0.075409,-0.32091,-0.216940,-0.20411,-0.336500,-0.282780,-0.213230,-0.292080,...,0.027988,-0.340940,-0.291220,3.314800,0.12707,0.298470,-0.266800,0.063750,3.55690,0.004834
9,-0.25832,0.70085,0.179340,-0.31544,-0.315110,0.53833,-0.027095,-0.302740,-0.218480,-0.152440,...,-0.159760,-0.320480,0.239730,0.043636,2.25280,0.507060,-0.307170,-0.180140,3.63510,-0.112150


X-fold validation with one file/sample being left out each iteration and used as test:
CV-samples in paper

In [13]:
labelfiles = os.listdir("../Data/PANORAMA/Labels/")
samplefiles = os.listdir("../Data/PANORAMA/Samples/")
labelfiles.sort()
samplefiles.sort()



## Stratify PANORAMA per sample
Train 10 iterations of PANORAMA with 9 samples, leaving one out <br>
Predict labels for sample which was excluded from training <br>
Save predictions to file and prediction accuracy to variable <br>
PANORAMA CV-samples in paper

In [14]:
kfoldsplitter = KFold(n_splits=10)
for train, test in kfoldsplitter.split(labelfiles):
    classifier = LinearDiscriminantAnalysis()
    for trainingset in train:
        X = pd.read_csv("../Data/PANORAMA/Samples/" + samplefiles[trainingset], header=None).values #train
        y = pd.read_csv("../Data/PANORAMA/Labels/" + labelfiles[trainingset], header=None).values.ravel() #labels
        classifier.fit(X, y)
    testdata = pd.read_csv("../Data/PANORAMA/Samples/" + samplefiles[test[0]], header=None).values #test
    testlabels = pd.read_csv("../Data/PANORAMA/Labels/" + labelfiles[test[0]], header=None).values.ravel() #testlabels
    #score testdata with its true labels in model trained against 9 other samples & save to file
    prediction = classifier.predict(testdata)
    with open('../Results/Predictions/PANORAMA LDA/CV-sample/predict_'+samplefiles[test[0]], 'w') as f:
        for item in prediction:
            f.write("%s\n" % item)
    print("Done testing %s" % samplefiles[test[0]])


Done testing Samples01.csv
Done testing Samples02.csv
Done testing Samples03.csv
Done testing Samples04.csv
Done testing Samples05.csv
Done testing Samples06.csv
Done testing Samples07.csv
Done testing Samples08.csv
Done testing Samples09.csv
Done testing Samples10.csv


## Stratify PANORAMA 5-fold

In [16]:
kfoldsplitter = KFold(n_splits=5)

#Load all samples and labels for 5-cold CV
samplesheap = pd.DataFrame()
labelsheap = pd.DataFrame()

for labelname, samplename in zip(labelfiles, samplefiles):
    samplesheap = samplesheap.append(pd.read_csv("../Data/PANORAMA/Samples/" + samplename, header=None))
    labelsheap = labelsheap.append(pd.read_csv("../Data/PANORAMA/Labels/" + labelname, header=None))



Number of predictions/trues matches original numer of instances

In [17]:


batch = 1
for train, test in kfoldsplitter.split(samplesheap):
    classifier = LinearDiscriminantAnalysis()
    #select test/train/true labels
    testdata = samplesheap.iloc[test].values
    traindata = samplesheap.iloc[train].values
    trainlabels = labelsheap.iloc[train].values.ravel()
    #fit & predict
    classifier.fit(traindata, trainlabels)
    prediction = classifier.predict(testdata)
    #write prediction to file
    print(len(prediction))

    with open('../Results/Predictions/PANORAMA LDA/CV-cells/predict_batch_' + str(batch) +'.csv', 'w') as f:
        for item in prediction:
            f.write("%s\n" % item)
    print("Done testing %s" % str(batch))
    batch += 1

102878
Done testing 1
102877
Done testing 2
102877
Done testing 3
102877
Done testing 4
102877
Done testing 5


In [18]:
#remove enormous datasets from memory
print(len(samplesheap))

del labelsheap
del samplesheap

514386
